In [1]:
import numpy as np
np.set_printoptions(precision=4)
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
import json
import sys
import os
import p_tqdm
from tqdm import tqdm
import json
import glob
import multiprocessing
from sqlalchemy import create_engine
sys.path.append('/inwdata2/Prithvi/GIT/analytics/')
%load_ext autoreload
%autoreload 2

/inwdata2/Prithvi/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# EEC analysis for 69k videos

## reading global csv data

In [2]:
global_mined_data = pd.read_csv('/inwdata2/Prithvi/GIT/work/AN25908/avid_uuid_s3_path.csv')
global_mined_data = global_mined_data.drop(columns=['alert_id', 'event_code', 's3_bucket']) # dropping unnecessary columns

# avid, uuid, avid_folder_name set to string
global_mined_data['avid'] = global_mined_data['avid'].astype(str)
global_mined_data['uuid'] = global_mined_data['uuid'].astype(str)
global_mined_data['avid_folder_name'] = global_mined_data['avid_folder_name'].astype(str)

# start_timestamp and end_timestamp to int
global_mined_data['start_timestamp'] = global_mined_data['start_timestamp'].astype(int)
global_mined_data['end_timestamp'] = global_mined_data['end_timestamp'].astype(int)

# formatting uuid to standard format
global_mined_data['uuid'] = global_mined_data['uuid'].apply(lambda temp_uuid: f"{temp_uuid[:8]}-{temp_uuid[8:12]}-{temp_uuid[12:16]}-{temp_uuid[16:20]}-{temp_uuid[20:]}") 
print(f"Global mined data shape: {global_mined_data.shape}")

Global mined data shape: (217040, 5)


## analysis for duration which are between 1700 to 2500

### reading annotation data

In [3]:
annotations_df = pd.read_csv('/inwdata2/Prithvi/GIT/work/AN25908/eec_annotations.csv')
print(f"Annotations dataframe shape: {annotations_df.shape}")
# annotations_df.set_index('uuid', inplace=True)
annotations_df.head()

Annotations dataframe shape: (6516, 11)


,uuid,start_frame,end_frame,Are_both_eyes_clearly_visible,Eyes_look_COMPLETELY_closed_due_to,Eyes_not_COMPLETELY_closed_then,Is_Camera_placement_valid?,Is_person_both_eyes_looks_COMPLETELY_closed_in_during_the_entire_alert_window?,Is_person_looking_outside_[INVALID_POSE]_or_upside?,Is_person_wearing_any_artifacts?,Is_there_any_obtstruction_or_reflection_in_any_eye?
0,000177ba-0925-4dda-95ac-0a2751b0a525,20.0000,38.0000,Yes,Not_applicable,Head_looking_sideways,Yes,No,No,Spects,No
1,00166eee-6e19-41a7-aef4-b45f43f82b0c,20.0000,43.0000,Yes,Fatigue,Not_Applicable,Yes,Yes,No,Not_Applicable,No
2,001b29c1-1d98-48c4-9ea6-4d5381774803,20.0000,42.0000,Yes,Not_applicable,Eyes_gaze_down,Yes,No,No,Not_Applicable,No
3,002cb457-3aa6-4d83-88ae-188f24250e39,20.0000,46.0000,Yes,Head_looking_sideways,Not_Applicable,Yes,Yes,Yes,Not_Applicable,No
4,00329be4-9142-42b0-a57b-3104003531aa,20.0000,39.0000,Yes,Not_applicable,Head_looking_sideways,Yes,No,No,Not_Applicable,No


### Merging annotation and global mined data

In [4]:
evaluation_data = pd.merge(annotations_df, global_mined_data, on='uuid', how='inner')
print(f"Evaluation data shape: {evaluation_data.shape}")
evaluation_data.head()

Evaluation data shape: (6516, 15)


,uuid,start_frame,end_frame,Are_both_eyes_clearly_visible,Eyes_look_COMPLETELY_closed_due_to,Eyes_not_COMPLETELY_closed_then,Is_Camera_placement_valid?,Is_person_both_eyes_looks_COMPLETELY_closed_in_during_the_entire_alert_window?,Is_person_looking_outside_[INVALID_POSE]_or_upside?,Is_person_wearing_any_artifacts?,Is_there_any_obtstruction_or_reflection_in_any_eye?,avid,avid_folder_name,start_timestamp,end_timestamp
0,000177ba-0925-4dda-95ac-0a2751b0a525,20.0000,38.0000,Yes,Not_applicable,Head_looking_sideways,Yes,No,No,Spects,No,b53d09c0-c0e5-4507-9edc-170bdb9b6ebb,b53d09c0-c0e5-4507-9edc-170bdb9b6ebb__34633118,17200,18900
1,00166eee-6e19-41a7-aef4-b45f43f82b0c,20.0000,43.0000,Yes,Fatigue,Not_Applicable,Yes,Yes,No,Not_Applicable,No,56663623-7978-47f4-b3ae-774d757f51ad,56663623-7978-47f4-b3ae-774d757f51ad__33324372,12400,14600
2,001b29c1-1d98-48c4-9ea6-4d5381774803,20.0000,42.0000,Yes,Not_applicable,Eyes_gaze_down,Yes,No,No,Not_Applicable,No,b2457224-e55d-45ef-be83-fc7e0b219fe9,b2457224-e55d-45ef-be83-fc7e0b219fe9__30867603,25700,27800
3,002cb457-3aa6-4d83-88ae-188f24250e39,20.0000,46.0000,Yes,Head_looking_sideways,Not_Applicable,Yes,Yes,Yes,Not_Applicable,No,491ee0a3-33d7-44ca-9f79-9aab6e86069b,491ee0a3-33d7-44ca-9f79-9aab6e86069b__41575093,57000,59500
4,00329be4-9142-42b0-a57b-3104003531aa,20.0000,39.0000,Yes,Not_applicable,Head_looking_sideways,Yes,No,No,Not_Applicable,No,c522794f-daa6-4e13-bb50-8c137acb4302,c522794f-daa6-4e13-bb50-8c137acb4302__32516910,53900,55700


In [5]:
from src.dms.modules.eec_with_obs import ExtendedEyeClosure

ImportError: py_nd_auth.a is not linked properly!! error: libpython3.9.so.1.0: cannot open shared object file: No such file or directory


WARNING - /home/ubuntu/config/deviceconfig.ini has no section identity
WARNING - Unsupported device type: None
Using mod_config: /home/ubuntu/.nddevice/nd_config_mod.ini
Using override_config: /home/ubuntu/config/bagheera_override.ini
Reading config: /inwdata2/Prithvi/GIT/analytics/src/nd_config.ini
WARNING - Cannot import label map for /home/ubuntu/autocam/tycho_yolo_usuk_v6.5.8
WARNING - This should only happen on the cloud side
WARNING - Cannot import label map for /home/ubuntu/autocam/tloc_US_v0.11.0.0.5
WARNING - This should only happen on the cloud side
WARNING - Cannot import label map for /home/ubuntu/autocam/tsc_v1.8.3.4
WARNING - This should only happen on the cloud side
WARNING - Cannot import label map for /home/ubuntu/autocam/rmc_v0.2.0.1.4
WARNING - This should only happen on the cloud side
WARNING - Cannot import label map for /home/ubuntu/autocam/pose_yolo_multihead_v2.0.5.4
WARNING - This should only happen on the cloud side
WARNING - Cannot import label map for /home/

In [6]:
def evaluate_algorithm(each_row, log_flag=True) -> pd.Series: # each row in evaluation_data

    def check_overlap(pred_start, pred_end, gt_start, gt_end) -> bool:
        return max(pred_start, gt_start) < min(pred_end, gt_end)
    
    json_path = f"/inwdata2/Prithvi/AN_25908_eec_recall_improvement/dms_submit_job_141184/{each_row['avid_folder_name']}/summary.json"
    with open(json_path, 'r') as f:
        json_dict = json.load(f)
    detections = json_dict['inference_data']['dms']['detections']
    eec_module = ExtendedEyeClosure(processfps = 10)
    eec_module.reset_state()
    for eachDetection in detections:
        start_ts = detections[0]['ts']
        event_obs = eec_module.update(lr_eye_detection_scores = (eachDetection['left_eye_det_scores'], eachDetection['right_eye_det_scores']),
                                      pose = eachDetection['pose_valid'],
                                      nose = eachDetection['nose'],
                                      face_bb = eachDetection['face_bbox'],
                                      ts = eachDetection['ts'] - start_ts,
                                      speed = eachDetection['speed'],
                                      latlong = (0,0),
                                      p_angle = eachDetection.get('p_angle', 0.0),
                                      y_angle = eachDetection.get('y_angle', 0.0))
        if log_flag:
            print(event_obs)
    events, obs_data, eec_end_event_obs = eec_module.serialize_for_session()
    if log_flag:
        print(obs_data['event_info'])
        print(events)
    ret_dict = {'uuid': each_row['uuid'], 
                'TP': 0, 'FP': 0, 'FN': 0, 'TN': 0, 
                'st_fidx': -1, 'et_fidx': -1, 
                'avid': each_row['avid']}
    for each_observation in obs_data['event_info']:
        overlap_flag = check_overlap(each_observation['st_fidx'], each_observation['et_fidx'],
                                 each_row['start_timestamp']//100, each_row['end_timestamp']//100)
        cond1 = overlap_flag == True
        cond2 = each_row['Is_person_both_eyes_looks_COMPLETELY_closed_in_during_the_entire_alert_window?'] == 'Yes'
        cond3 = each_observation['is_valid_eec'] == True
        cond4 = (each_row['Eyes_look_COMPLETELY_closed_due_to'] == 'Microsleep') or (each_row['Eyes_look_COMPLETELY_closed_due_to'] == 'Fatigue')
        if cond1 and cond2 and cond3:
            ret_dict['TP'] += 1
            ret_dict['st_fidx'] = each_observation['st_fidx']
            ret_dict['et_fidx'] = each_observation['et_fidx']
        elif cond1 and (not cond2) and cond3:
            ret_dict['FP'] += 1
            ret_dict['st_fidx'] = each_observation['st_fidx']
            ret_dict['et_fidx'] = each_observation['et_fidx']
    if ret_dict['TP'] == 0 and each_row['Is_person_both_eyes_looks_COMPLETELY_closed_in_during_the_entire_alert_window?'] == 'Yes':
        ret_dict['FN'] += 1
        ret_dict['st_fidx'] = each_row['start_timestamp']//100
        ret_dict['et_fidx'] = each_row['end_timestamp']//100
    if ret_dict['FP'] == 0 and each_row['Is_person_both_eyes_looks_COMPLETELY_closed_in_during_the_entire_alert_window?'] == 'No':
        ret_dict['TN'] = 1
        ret_dict['st_fidx'] = each_row['start_timestamp']//100
        ret_dict['et_fidx'] = each_row['end_timestamp']//100

    return pd.Series(ret_dict)
    

In [7]:
evaluation_data[evaluation_data['uuid'] == 'cc7f2da9-5f76-4b6e-8fe2-03b100accb3e']

,uuid,start_frame,end_frame,Are_both_eyes_clearly_visible,Eyes_look_COMPLETELY_closed_due_to,Eyes_not_COMPLETELY_closed_then,Is_Camera_placement_valid?,Is_person_both_eyes_looks_COMPLETELY_closed_in_during_the_entire_alert_window?,Is_person_looking_outside_[INVALID_POSE]_or_upside?,Is_person_wearing_any_artifacts?,Is_there_any_obtstruction_or_reflection_in_any_eye?,avid,avid_folder_name,start_timestamp,end_timestamp
5584,cc7f2da9-5f76-4b6e-8fe2-03b100accb3e,5.0000,24.0000,Yes,Eyes_gaze_Away,Not_Applicable,Yes,Yes,No,Not_Applicable,No,a36a0546-8d93-4dd8-afde-b0739018eb47,a36a0546-8d93-4dd8-afde-b0739018eb47__35648058,500,2300


In [8]:
temp = evaluate_algorithm(evaluation_data[evaluation_data['uuid'] == 'ff8aa7d8-24f4-4304-bd0d-ed9b3c33c8f8'].to_dict('records')[0])

{'status': False, 'st_time': None, 'et_time': None, 'start_latlong': None, 'end_latlong': None, 'st_fidx': -1, 'et_fidx': -1, 'cur_fidx': -1, 'process_trigger': 0, 'eye_status': 0}
{'status': False, 'st_time': None, 'et_time': None, 'start_latlong': None, 'end_latlong': None, 'st_fidx': -1, 'et_fidx': -1, 'cur_fidx': -1, 'process_trigger': 0, 'eye_status': 0}
{'status': False, 'st_time': None, 'et_time': None, 'start_latlong': None, 'end_latlong': None, 'st_fidx': -1, 'et_fidx': -1, 'cur_fidx': -1, 'process_trigger': 0, 'eye_status': 1}
{'status': False, 'st_time': None, 'et_time': None, 'start_latlong': None, 'end_latlong': None, 'st_fidx': -1, 'et_fidx': -1, 'cur_fidx': -1, 'process_trigger': 0, 'eye_status': -1}
{'status': False, 'st_time': None, 'et_time': None, 'start_latlong': None, 'end_latlong': None, 'st_fidx': -1, 'et_fidx': -1, 'cur_fidx': -1, 'process_trigger': 0, 'eye_status': 0}
{'status': False, 'st_time': None, 'et_time': None, 'start_latlong': None, 'end_latlong': None

In [9]:
results = p_tqdm.p_map(lambda row: evaluate_algorithm(row, log_flag=False), evaluation_data.to_dict(orient='records'), num_cpus=multiprocessing.cpu_count(), desc='Evaluating EEC algorithm on evaluation data')
results_df = pd.DataFrame(results)
results_df.shape

Evaluating EEC algorithm on evaluation data: 100%|██████████| 6516/6516 [00:31<00:00, 208.96it/s]


(6516, 8)

In [10]:
recall = results_df['TP'].sum() / (results_df['TP'].sum() + results_df['FN'].sum())
precision = results_df['TP'].sum() / (results_df['TP'].sum() + results_df['FP'].sum())
print("EEC Algorithm Evaluation Results:")
print(f"True Positives: {results_df['TP'].sum()}, False Positives: {results_df['FP'].sum()}, False Negatives: {results_df['FN'].sum()}, True Negatives: {results_df['TN'].sum()}")
print(f"Recall: {recall:.4f}, Precision: {precision:.4f}")

EEC Algorithm Evaluation Results:
True Positives: 9, False Positives: 1, False Negatives: 2585, True Negatives: 3918
Recall: 0.0035, Precision: 0.9000


In [11]:
results_df[results_df['TP'] > 0]

,uuid,TP,FP,FN,TN,st_fidx,et_fidx,avid
11,0056d80a-c7b5-4d7b-b092-5bf0d6822e90,1,0,0,0,49,66,7bfdbc02-52d8-4df8-a405-a88332823ec4
1489,25e2a970-f40f-446d-a4ce-d203d15b1aba,1,0,0,0,492,509,1610e323-314d-42ab-9d3c-ec4356280f38
1924,31b0c0e7-2dbc-4a35-8155-cc26c3307c75,1,0,0,0,568,588,a731b8b0-86d9-4e07-b940-477faea37970
1993,332c75f0-a166-44e1-a803-8af04ecb4c06,1,0,0,0,473,490,4047fa8b-b0d7-4ac6-b248-f7cf6b5894b1
2878,4b599798-ba34-493e-bf79-52aa4556733b,1,0,0,0,346,365,3e23129a-a4f4-4e2b-90e5-8ca1e3d851cc
2936,4ccb54b8-9f1e-4449-9c74-920ca0afb0f8,1,0,0,0,356,374,162161a5-8747-4567-9d97-2336deab8839
4418,8cc2dae3-a296-4658-aa5b-6ab5a6adf4ca,1,0,0,0,94,113,1db77cb3-8579-4a77-a401-bbe3b9925171
4503,91a0f364-ce2a-41a4-81c2-f9518bb00e6c,1,0,0,0,209,226,a3c64440-4b69-4aa5-8ae3-0c746f59a245
4835,a26f5e19-45fd-4ad5-a27a-d79a18352f32,1,0,0,0,528,548,2fb97ea2-d632-4c4a-af33-f40ab9d1ccef


In [12]:
results_df[results_df['FN'] > 0][-25:]

,uuid,TP,FP,FN,TN,st_fidx,et_fidx,avid
6404,f9b4ac48-9149-415e-a894-73d5af5257fb,0,0,1,0,53,72,1cf6c95c-84f6-4e4f-a1d7-b2ee234ac7da
6416,fa72a6b9-a167-42ee-8298-86ac609870b5,0,0,1,0,486,505,05e64784-eba1-43bd-b082-23fe38fb2620
6418,fa84186e-04b0-4040-b40d-19d178fa30a7,0,0,1,0,414,438,cdea0c1f-1f92-4d96-ad6c-9cd430c3eba5
6426,fb0f22b1-a6f7-491e-984e-613dddd1bf50,0,0,1,0,149,172,255aae9e-aa49-41c3-ad56-4d798de9e433
6427,fb0fe85c-ab73-4f87-aa50-0ba5694ab4b3,0,0,1,0,70,89,597cfc96-98d6-44f9-9832-b68b7009f44b
6447,fc5432fc-e01a-4c4f-b6b3-c1731029c543,0,0,1,0,0,23,ae2e9a77-0cb7-4ae0-8a2e-36c340e6847b
6455,fcc7c970-02e7-4875-b2b9-0d375a881339,0,0,1,0,542,559,9381493c-66e1-4ded-b518-186204b846bd
6456,fcc8a79e-d1f7-4985-9334-84fd6d41c890,0,0,1,0,213,232,9a786ebf-0d6a-48cf-ba6e-8768a402b1a8
6457,fcd8adf3-bd0b-4de7-97f4-daecfdf34112,0,0,1,0,435,459,243661ef-bfd0-4585-b74d-c6b6e99b07d7
6463,fd5dc376-08ca-4365-be81-6879d2ce3fac,0,0,1,0,340,357,800c54ef-5243-4582-83bc-b56171e75d8a


In [13]:
temp = 'ff8aa7d8-24f4-4304-bd0d-ed9b3c33c8f8'